In [1]:
!pip install datasets


In [2]:
pip install --upgrade transformers


In [3]:
from transformers import Trainer
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score
import multiprocessing

# Задайем название модели, которую хотим загрузить с Hugging Face
model_name = "DeepPavlov/rubert-base-cased"  # Заменим на любую другую модель из Hugging Face
num_labels = 3  # Укажим количество классов для вашей задачи (меняется в зависимости от датасета)

# Загрузка датасета с Hugging Face (можно также изменить на любой другой набор данных)
dataset = load_dataset("MonoHime/ru_sentiment_dataset")

# Получаем тренировочный и валидационный наборы данных
df_train = dataset['train'].to_pandas()
df_val = dataset['validation'].to_pandas()

# Проверка уникальных значений в колонке sentiment
print("Уникальные значения в тренировочном наборе:", df_train['sentiment'].unique())

# Загружаем выбранную модель и токенизатор
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Функция токенизации
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Преобразуем pandas DataFrame в Dataset для Hugging Face
def convert_to_dataset(df, text_column="text", label_column=None):
    dataset = Dataset.from_pandas(df)
    if label_column:
        dataset = dataset.rename_column(label_column, "labels")  # Переименовываем метку
    dataset = dataset.map(tokenize_function, batched=True, num_proc=multiprocessing.cpu_count())  # Параллельная токенизация
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'] if label_column else ['input_ids', 'attention_mask'])
    return dataset

# Преобразуем данные в Dataset
train_dataset = convert_to_dataset(df_train, text_column="text", label_column="sentiment")
val_dataset = convert_to_dataset(df_val, text_column="text", label_column="sentiment")

# Метрика качества
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)  # Преобразуем logits в тензор
    predictions = torch.argmax(logits, dim=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

# Предсказания до дообучения
trainer = Trainer(model=model)
predictions = trainer.predict(val_dataset)
accuracy_before_finetuning = accuracy_score(predictions.predictions.argmax(axis=-1), predictions.label_ids)
print(f"Accuracy до дообучения: {accuracy_before_finetuning}")

# Вывод нескольких примеров предсказаний
num_examples_to_show = 10  # Выводим 10 примеров
for i in range(num_examples_to_show):
    text = df_val.iloc[i]['text']  # Текст твита
    real_label = df_val.iloc[i]['sentiment']  # Реальный лейбл
    predicted_label = predictions.predictions.argmax(axis=-1)[i]  # Предсказанный лейбл

    print(f"Текст: {text}")
    print(f"Реальный класс: {real_label}")
    print(f"Предсказанный класс: {predicted_label}")
    print("-" * 50)

# Настройки для дообучения модели
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,
    gradient_accumulation_steps=4,
    dataloader_num_workers=multiprocessing.cpu_count(),
    save_strategy="epoch"  # Изменение save_strategy
)

# Trainer для обучения
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Обучаем модель
trainer.train()

# Предсказания после дообучения
predictions_after = trainer.predict(val_dataset)
accuracy_after_finetuning = accuracy_score(predictions_after.predictions.argmax(axis=-1), predictions_after.label_ids)
print(f"Accuracy после дообучения: {accuracy_after_finetuning}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Уникальные значения в тренировочном наборе: [1 2 0]


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map (num_proc=2):   0%|          | 0/189891 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/21098 [00:00<?, ? examples/s]

Accuracy до дообучения: 0.27533415489619867
Текст: Развода на деньги нет Наблюдаюсь в Лайфклиник по беременности, развода на деньги нет, врачи не плохие, по поводу ресепшен соглашусь (путают документы). 
Реальный класс: 1
Предсказанный класс: 2
--------------------------------------------------
Текст: Отель выбрали потому что рядом со стадионом. Отель 4*. Номер большой. Кровать 2-спальная одна. 2 одеяла. Много подушек. Есть зона отдыха. Чайный сет. 2 бутылки по 0,5 л бесплатно каждый день. Много шкафов. Мини-бар. Утюг и гладильная доска, отдельно прибор для глажки брюк. Санузел общий большой. Ванна, лейка съемная. 2 раковины, фен. Халаты, тапочки. Расширенный пакет косметических средств, даже соль для ванны. Интернет быстрый. Так как этаж высокий был, вид из окна на город. Один недостаток в номере 919: он напротив хозлифта и с утра начинает пользоваться им персонал для обслуживания завтрака на 9 этаже. Очень слышно звон посуды и разговоры. Завтрак не включен, стоит 23 евро. Рядом с оте

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
0,0.418500,0.438219,0.802304
1,0.321600,0.432494,0.810930
2,0.280700,0.458315,0.814864


Accuracy после дообучения: 0.8148639681486397


Как видно из кода,что Accuracy до дообучения: 0.27533415489619867
было не очень болшое, а Accuracy после дообучения: 0.8148639681486397 и это значение будет приемлеммым для дальнейшего использования.